In [ ]:
import os
from os.path import join, isdir, isfile, abspath, dirname, splitext, basename, split
from parse import parse
import math

from IPython.display import Markdown, display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from misc_util.pretty_print import pretty_print as print
from derive_conceptualspace.pipeline import CustomContext, SnakeContext
from derive_conceptualspace.settings import ENV_PREFIX, get_setting
from derive_conceptualspace import settings
from derive_conceptualspace.pipeline import load_envfiles

plt.rcParams['figure.figsize'] = [16, 10]

In [ ]:
def show_metrics(metlist):
    metrics = list(list(metlist.values())[0].keys())
    w, h = math.ceil(math.sqrt(len(metrics))), math.ceil(math.sqrt(len(metrics)))
    for n, met in enumerate(metrics):
        plt.subplot(w, h, n+1)
        vals = [i[met] for i in metlist.values()]
        vals = [abs(i) for i in vals]
        #vals = (np.digitize(vals, [i/10 for i in arange(11)])-1)/10
        hist = np.histogram(vals, bins=[i/10 for i in range(11)]+[max(vals+[1.1])])
        make_logscale = max(hist[0])/len(vals) > 0.6
        plt.title(met +(" (log)" if make_logscale else ""))
        plt.hist(vals, bins=[i/10 for i in range(11)]+[max(vals+[1.1])], log=make_logscale, color="red" if make_logscale else "blue")
    plt.tight_layout()
    plt.show()

## Prepare all parameter-combinations

In [ ]:
load_envfiles("placetypes")

candidates = [join(path, name)[len(os.environ["MA_BASE_DIR"])+1:] for path, subdirs, files in os.walk(join(os.environ["MA_BASE_DIR"], "")) for
              name in files if name.startswith("clusters.json")]
candidates = [i for i in candidates if i.startswith(os.environ["MA_DATASET"]) and not "backup" in i.lower() and not "debug_True" in i]
print(f"There are {len(candidates)} different parameter-combis for dataset *b*{os.environ['MA_DATASET']}*b*:")
# display(candidates)
configs = [parse(os.sep.join(settings.DIR_STRUCT+["clusters.json"]), cand).named for cand in candidates]
print_cnf = {k: list(set(dic[k] for dic in configs)) for k in configs[0]}
print_cnf = {k: v[0] if len(v) == 1 else v for k, v in print_cnf.items()}
print_cnf

# Showing all metrics for the loaded parameter-combinations

In [ ]:
# for conf in configs:
#     specials = {k: v for k, v in conf.items() if isinstance(print_cnf[k], list)}
#     display(Markdown("## "+", ".join(f"{k}: {v}" for k,v in specials.items())))
#     ctx = SnakeContext.loader_context(config=conf, silent=True)
#     clusters = ctx.load("clusters")
#     #show_metrics(clusters)
#     display(generate_comparertable(clusters["metrics"], minval=0.1))
#     #look at combis o'f bin2bin, f_one, k_r2r+_min, k_dig+_2, k_c2r+

In [ ]:
for conf in configs:
    specials = {k: v for k, v in conf.items() if isinstance(print_cnf[k], list)}
    display(Markdown("## "+", ".join(f"{k}: {v}" for k,v in specials.items())))
    ctx = SnakeContext.loader_context(config=conf, silent=True)
    clusters = ctx.load("clusters")
    ctx.print_important_settings()
    show_metrics(clusters["metrics"])

In [ ]:
lambda1 = 0.4
lambda2 = 0.1

for conf in configs:
    specials = {k: v for k, v in conf.items() if isinstance(print_cnf[k], list)}
    display(Markdown("#### "+", ".join(f"{k}: {v}" for k,v in specials.items())))
    ctx = SnakeContext.loader_context(config=conf, silent=True)
    clusters = ctx.load("clusters")
    metlist = clusters["metrics"]
    for met in list(list(metlist.values())[0].keys()):
        if "kappa" in met and not "bin2bin" in met:
            vals = [i[met] for i in metlist.values()]
            t1 = len([i for i in vals if i >= lambda1])
            t2 = len([i for i in vals if i >= lambda2]) - t1
            if t1: 
                print(f" {met}: T^{lambda1}: {t1}, T^{lambda2}: {t2}")
                print(f" In T^{lambda1}: {', '.join([k for k, v in metlist.items() if v[met] > lambda1])}")